### Signature Verification Model using Vgg16 and CNN-Seq

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
#for files selection
import glob

gen = [glob.glob('data/Dataset_Signature_Final/Dataset/dataset1/real/*.*'),
       glob.glob('data/Dataset_Signature_Final/Dataset/dataset2/real/*.*'),
       glob.glob('data/Dataset_Signature_Final/Dataset/dataset3/real/*.*'),
       glob.glob('data/Dataset_Signature_Final/Dataset/dataset4/real1/*.*')]
                 
forg = [glob.glob('data/Dataset_Signature_Final/Dataset/dataset1/forge/*.*'),
        glob.glob('data/Dataset_Signature_Final/Dataset/dataset2/forge/*.*'),
        glob.glob('data/Dataset_Signature_Final/Dataset/dataset3/forge/*.*'),
        glob.glob('data/Dataset_Signature_Final/Dataset/dataset4/forge/*.*')]

In [4]:
import cv2
from sklearn.model_selection import train_test_split

train_data = []
train_labels = []
test_data = []
test_labels = []

#Gen and forg are lists containing image paths
all_data = []  # Combine all images
all_labels = []  # Combine all labels

for data in gen:
    for i in data:
        image = cv2.imread(i)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        all_data.append(image)
        all_labels.append(0)  # Genuine = 0

for data in forg:
    for j in data:
        image = cv2.imread(j)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        all_data.append(image)
        all_labels.append(1)  # Forged = 1

# Split data into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(all_data, all_labels, test_size=0.2, random_state=42)

# Convert to NumPy arrays
train_data = np.array(X_train) / 255.0
train_labels = np.array(y_train)
test_data = np.array(X_test) / 255.0
test_labels = np.array(y_test)


In [5]:
train_data.shape

(576, 224, 224, 3)

In [6]:
test_data.shape

(144, 224, 224, 3)

In [7]:
test_labels.shape

(144,)

In [8]:
import tensorflow as tf
tf.__version__

'2.16.1'

### Vgg16 Pre Trained Model

In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score


# Define image dimensions
img_width, img_height = 224, 224

# Data normalization
train_data = train_data.astype('float32') / 255.0
test_data = test_data.astype('float32') / 255.0

# Preprocessing for VGG16 
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_data,train_labels,  
        # target_size=(img_width, img_height),
        batch_size=32
        # class_mode='binary'  
)


test_generator = test_datagen.flow(test_data,test_labels, 
        # target_size=(img_width, img_height),
        batch_size=32  
        # class_mode='binary'
)


# Load the VGG16 model (exclude the top layers)
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze pre-trained VGG16 layers, here we made it True for better results
for layer in vgg16_model.layers:
    layer.trainable = True

# Add custom layers for classification
x = vgg16_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)  

# Create the final model
model = Model(inputs=vgg16_model.input, outputs=x)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy','precision', 'recall'])

# Early stopping 
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

# Adjust batch size and epochs as needed
model.fit(train_data, train_labels, epochs=10, batch_size=16, validation_split=0.2, callbacks=[early_stopping])


# Make predictions on test data
predictions = model.predict(test_data)

# Threshold predictions 
predictions = (predictions > 0.5).astype(int)  

print('Predictions:', predictions)

test_loss, test_acc, test_precision, test_recall = model.evaluate(test_data, test_labels)

print('Test Accuracy:', test_acc)
print('Test Precision:', test_precision)
print('Test Recall:', test_recall)

Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 297s 10s/step - accuracy: 0.4591 - loss: 0.7845 - precision: 0.4289 - recall: 0.4497 - val_accuracy: 0.4914 - val_loss: 0.7309 - val_precision: 0.4914 - val_recall: 1.0000
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 289s 10s/step - accuracy: 0.5157 - loss: 0.7086 - precision: 0.5248 - recall: 0.7048 - val_accuracy: 0.4914 - val_loss: 0.7059 - val_precision: 0.4914 - val_recall: 1.0000
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 293s 10s/step - accuracy: 0.5820 - loss: 0.6913 - precision: 0.6033 - recall: 0.8060 - val_accuracy: 0.4914 - val_loss: 0.6960 - val_precision: 0.4914 - val_recall: 1.0000
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 288s 10s/step - accuracy: 0.4801 - loss: 0.6933 - precision: 0.4901 - recall: 0.6303 - val_accuracy: 0.4914 - val_loss: 0.6935 - val_precision: 0.4914 - val_recall: 1.0000
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 292s 10s/step - accuracy: 0.5523 - loss: 0.6912 - precision: 0.5611 - recall: 0.9569 - val_accuracy: 0.4914 - val_loss: 0.

### Vgg16 using SGD Optimizer

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras.optimizers import SGD
from keras import layers
from keras import regularizers


# Define image dimensions
img_width, img_height = 224, 224

# Data normalization 
train_data = train_data.astype('float32') / 255.0
test_data = test_data.astype('float32') / 255.0

# Preprocessing for VGG16, few more pre processors added  
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True,
                                    channel_shift_range=50,rotation_range=20,width_shift_range=0.2,
                                    height_shift_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_data,train_labels,  
        # target_size=(img_width, img_height),
        batch_size=32  
        # class_mode='binary' 
)


test_generator = test_datagen.flow(test_data,test_labels,  
        # target_size=(img_width, img_height),
        batch_size=32 
        # class_mode='binary' 


# Load the VGG16 model 
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze pre-trained VGG16 layers 
for layer in vgg16_model.layers:
    layer.trainable = True

# Add custom layers for classification
x = vgg16_model.output
x = Flatten()(x)
#x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x) 
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x) 

# Create the final model
model = Model(inputs=vgg16_model.input, outputs=x)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.001,momentum=0.9), metrics=['accuracy','precision', 'recall'])

# Early stopping (optional)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

# Train with your data (adjust batch size and epochs as needed)
model.fit(train_data, train_labels, epochs=10, batch_size=16, validation_split=0.2, callbacks=[early_stopping])


# Make predictions on test data
predictions = model.predict(test_data)

# Threshold predictions 
predictions = (predictions > 0.5).astype(int) 

print('Predictions:', predictions)

test_loss, test_acc, test_precision, test_recall = model.evaluate(test_data, test_labels)

print('Test Accuracy:', test_acc)
print('Test Precision:', test_precision)
print('Test Recall:', test_recall)

Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 282s 10s/step - accuracy: 0.4476 - loss: 0.9101 - precision: 0.4409 - recall: 0.4637 - val_accuracy: 0.4914 - val_loss: 0.7061 - val_precision: 0.4914 - val_recall: 1.0000
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 276s 10s/step - accuracy: 0.5189 - loss: 0.6978 - precision: 0.5326 - recall: 0.8369 - val_accuracy: 0.4914 - val_loss: 0.6942 - val_precision: 0.4914 - val_recall: 1.0000
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 288s 10s/step - accuracy: 0.4995 - loss: 0.6940 - precision: 0.4963 - recall: 0.6695 - val_accuracy: 0.4914 - val_loss: 0.7086 - val_precision: 0.4914 - val_recall: 1.0000
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 313s 11s/step - accuracy: 0.5589 - loss: 0.6912 - precision: 0.5550 - recall: 0.8389 - val_accuracy: 0.4914 - val_loss: 0.6938 - val_precision: 0.4914 - val_recall: 1.0000
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 275s 9s/step - accuracy: 0.5519 - loss: 0.6900 - precision: 0.5461 - recall: 0.8480 - val_accuracy: 0.4914 - val_loss: 0.6

In [18]:
!pip install Keras-Preprocessing

Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)


### CNN - Relu

In [23]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from keras_preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix as CM
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard


def relu(x):
    return np.maximum(0, x)

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)

def objective(x, y):
 return x**2.0 + y**2.0


network = Sequential()

network.add(Conv2D(64,(3,3),input_shape=(224,224,3),activation='relu'))
network.add(MaxPooling2D(3,3))

network.add(Conv2D(32,(3,3),activation='relu'))
network.add(MaxPooling2D(2,2))

network.add(Flatten()) #NX1
network.add(Dense(128,activation = 'relu'))
network.add(Dropout(rate=0.3)) #143

network.add(Dense(2,activation = 'softmax'))

network.compile(optimizer=Adam(learning_rate=0.001),loss="binary_crossentropy",metrics=['accuracy','precision', 'recall'])
network.summary()


early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
callback_early_stop_reduceLROnPlateau=[early_stopping]
model.fit(train_data, train_labels, epochs=10, batch_size=16, validation_split=0.2, callbacks=[early_stopping])

predictions = model.predict(test_data)
predictions = (predictions > 0.5).astype(int) 

print('Predictions:', predictions)
test_loss, test_acc, test_precision, test_recall = model.evaluate(test_data, test_labels)
print('Test Accuracy:', test_acc)
print('Test Precision:', test_precision)
print('Test Recall:', test_recall)


c:\Users\nikil\anaconda3\envs\pytorch\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 222, 222, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 74, 74, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 72, 72, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 36, 36, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 41472)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │     5,308,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,329,058 (20.33 MB)

 Trainable params: 5,329,058 (20.33 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
27/29 ━━━━━━━━━━━━━━━━━━━━ 19s 10s/step - accuracy: 0.5322 - loss: 0.6910 - precision: 0.5322 - recall: 1.0000

In [17]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
earlyStopping = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=3,
                              verbose=1)

callback_early_stop_reduceLROnPlateau=[earlyStopping]


EPOCHS = 10
BS = 64
progess = model.fit(train_data,train_labels, batch_size=BS,epochs=EPOCHS, callbacks=callback_early_stop_reduceLROnPlateau,validation_split=.2)


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 314s 39s/step - accuracy: 0.5268 - loss: 0.6917 - precision: 0.5268 - recall: 1.0000 - val_accuracy: 0.4914 - val_loss: 0.6955 - val_precision: 0.4914 - val_recall: 1.0000
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 314s 39s/step - accuracy: 0.5310 - loss: 0.6914 - precision: 0.5310 - recall: 1.0000 - val_accuracy: 0.4914 - val_loss: 0.6954 - val_precision: 0.4914 - val_recall: 1.0000
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 320s 40s/step - accuracy: 0.4973 - loss: 0.6948 - precision: 0.4973 - recall: 1.0000 - val_accuracy: 0.4914 - val_loss: 0.6954 - val_precision: 0.4914 - val_recall: 1.0000
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 321s 40s/step - accuracy: 0.4975 - loss: 0.6946 - precision: 0.4975 - recall: 1.0000 - val_accuracy: 0.4914 - val_loss: 0.6953 - val_precision: 0.4914 - val_recall: 1.0000
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 323s 40s/step - accuracy: 0.5199 - loss: 0.6925 - precision: 0.5199 - recall: 1.0000 - val_accuracy: 0.4914 - val_loss: 0.6953 - val

In [18]:
acc = progess.history['accuracy']
val_acc = progess.history['val_accuracy']
loss = progess.history['loss']
val_loss = progess.history['val_loss']
 